## How Decision Trees Learn to Make Decisions
+ To understand how decision Trees Work, the concept of entropy

Information entropy is the average rate at which information is produced by a stochastic source of data.

The measure of information entropy associated with each possible data value is the negative logarithm of the probability mass function for the value:
+ for a single Class

    ent = -p*log(p) 
    
+ sum for all Classes 

 S = − ∑ -p*log(p) 

##### Function Descriptions
+ Entropy Function takes a probabity and return the entropy
+ homogeneity function takes and array and calculated the entropy of the array


In [6]:
#### Define a entropy function for a single binary class  (0 /1 ) outcome
import numpy as np
import pandas as pd
from collections import Counter

def get_entropy(p):
    if p == 0:
        return 0
    else:
        return -p * np.log2(p)
 

def get_homogeneity(x):
    n = len(x)
    counts = Counter(x).most_common()
    p = counts[0][1]/n
    return get_entropy(p) + get_entropy(1-p)


#### Explore How Entropy works on different
Entropy is measure between zero and one that measures how mix a variable is.
+ contast variables have a entropy of zero
+ 50/50 mix has the highest entropy
This demonstrates that entropy is highest then variables are perfectly mixed

In [7]:
# Contant Distrobutions 
print('entropy all ones', get_homogeneity(np.ones(100)))
# Contant Distrobutions 
print(' all zeros', get_homogeneity(np.zeros(100)))
print('half ones and zeros',get_homogeneity([0,1] * 100))
print('Mostly zeros', get_homogeneity([0, 0, 0, 1] * 100 )    )                        
print('Mostly Ones', get_homogeneity([1, 1 ,0, 1] * 100 )    )                                             

entropy all ones 0.0
 all zeros 0.0
half ones and zeros 1.0
Mostly zeros 0.8112781244591328
Mostly Ones 0.8112781244591328


#### Information Gain is the Difference in Entropy of a Given Variable before and after a split
This creates some data with p(y|x=1) = .25 and p(y|x=2) = .75

+ This creates a function to estimate information gain, using difference in homogeneity for each value, for y given x
+ This is typically used a split criteria in decision Trees
+ The higher the information gain the better the split
+ most of the time, frequency weighting is used to gaurd against over fitting.



In [9]:

x =  [1,1,1,1,2,2,2,2]
y = [0,0,0,1, 0,0,1,1]
get_information_gain(x,y)

{1: 0.1431558784658321, 2: -0.04556599707503506}

In [29]:
def get_information_gain(x, y):
    x = np.array(x)
    y = np.array(y)
    n = len(y)
    counts = Counter(x).most_common()
    p = counts[0][1]/n
    entropy_all = get_entropy(p)
    if len(counts) == 1:
        return entropy_all
    else:
        for val, c in counts:
            y_hat = y[x == val]
            
            w = len(y_hat)/n # calcuated the wieght
            p_hat = c[0][1]/len(y_hat)

            info_gain = entropy_all -  w * get_entropy(p_hat)
            output.update({c[0]:info_gain})
        return output



In [32]:
x =  [1,1,1,1,2,2,2,2]
y = [0,0,0,1, 0,0,1,1]
get_information_gain(x,y)

{1: 0.1431558784658321, 2: -0.04556599707503506}

### Gini Impurity 
An Alernate way to determin splits is to use Gini Imputiry gain
+ Gini Imputurity is as follows

+ Gini Impurity is the probability of incorrectly classifying a randomly chosen element in the dataset if it were 
randomly labeled according to the class distribution in the dataset. G=1∑p(i)∗(1−p(i))
+ Gini Gain of .5 suggest a perfect split

In [33]:
def get_impurity(x):
    counts = Counter(x).most_common() 
    n = len(x)
    if len(counts) == 1:
        p = counts[0][1]/n
        return p * (1-p)  +  (1-p) * (1 - (1- p))
    else:
        output= []
        for c in counts:
            p = c[1]/n
            output.append(p * (1-p) )
    return sum(output)


In [12]:
print(' all zeros', get_impurity(np.zeros(100)))
print('half ones and zeros',get_impurity([0,1] * 100))
print('Mostly zeros', get_impurity([0, 0, 0, 1] * 100 )    )                        
print('Mostly Ones', get_impurity([1, 1 ,0, 1] * 100 )    ) 

 all zeros 0.0
half ones and zeros 0.5
Mostly zeros 0.375
Mostly Ones 0.375


Gini Gain Function, returns and unweight gini gain of the split, (.5 is perfect and higher is better)

In [34]:
def get_gini_gain(x, y):
    x = np.array(x)
    y = np.array(y)
    n = len(y)
    counts = Counter(x).most_common()
    g = get_impurity(y)
    output = {}
    for c in counts:
        g_after =  get_impurity(y[x == c[0]])
        w = c[1]/n # calcuated the wieght
        gini_gain = g -   g_after
        output.update({c[0]:gini_gain })
    return output

    
    
    

In [35]:

x =  [1,1,1,1,2,2,2,2]
y = [0,0,0,1, 0,0,1,1]
get_gini_gain(x,y)

{1: 0.09375, 2: -0.03125}

#### Discritizing
Most Decision Trees havesome sort of discretizing (binning) method to handel continous varaibles
+ Equal Size Binning (same number of elements in each bin)
+ Equal Width Binning (each bin is has the same size range between start and end points)



In [70]:
from sklearn.preprocessing import KBinsDiscretizer as KBinsDiscretizer
d = np.random.normal(loc=100, scale = 3,size = (100, 2))
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
est.fit(d)
print('edges', est.bin_edges_)
est.transform(d[[1, 65, 75, 99], :])


edges [array([ 91.97987806,  95.94651973,  99.91316141, 103.87980308,
       107.84644475])
 array([ 90.78589642,  94.58096721,  98.37603799, 102.17110877,
       105.96617956])]


array([[2., 3.],
       [1., 2.],
       [2., 2.],
       [2., 1.]])

In [69]:
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
est.fit(d)
print('edges', est.bin_edges_)
est.transform(d[[1, 65, 75, 99], :])




edges [array([ 97.32589263,  99.32131666, 100.11349301, 100.7940867 ,
       102.11009358])
 array([ 97.6601149 ,  99.32039748, 100.12963615, 100.79573481,
       102.26733613])]


array([[3., 3.],
       [3., 1.],
       [1., 3.],
       [0., 0.]])